In [ ]:
import library.dataset_atlas as dataset_atlas
import library.dataset as dataset
import library.stats.atlas as atlas
# iPython-specific
import IPython

# standard libs
import pandas as pd
from pprint import pprint
from datetime import datetime, timedelta
from influxdb import DataFrameClient
import collections
import numpy as np
import math

# custom libs
import library.date_manipulation as date_manipulation
import library.stats.atlas as atlas
import library.stats.variation as stats_variation
import library.influx.query as influx_query

# dataset import
import library.dataset as dataset
#import for calculat time of execution 
import time

In [ ]:
start_time = time.time()

#Parametres for Getting the table from influxdb
boundary_from = '2018-06-13'
boundary_to = '2018-06-14'
table="timer_request_by_business"

#geting the table 

#df_dict = dataset.get_data_from_prod_interval_high_precision_for_atlas(boundary_from,boundary_to,table )
#df_dict = dataset.get_data_from_prod_interval_low_precision_sum_count_service_mno(boundary_from,boundary_to,table )

#add +2h for the local Time
#df_dict = dataset.get_data_from_prod_from_now_high_precision_for_atlas(6,table )
df_dict = dataset.get_data_from_prod_from_now_high_precision_for_atlas_by_service(50,table )


#Parametres for cleaning the series
period_between_points=10
period_between_points_unit="s"



#cleaning the dataframe's dict from series without enough points
df_dict=date_manipulation.remove_series_without_enough_points(boundary_from,boundary_to,df_dict,period_between_points, period_between_points_unit)


In [ ]:
#Parametres for Atlas

# possible values for prediction_method= mean \ least_squares \ polyfit
prediction_method='mean'

# possible values for keep = False \ True
keep=False
#field used for calculating os surprises in df_dict
field='count'
#interval used 
atlas_interval_duration=timedelta(hours=0.1)
atlas_sliding_shift=timedelta(minutes=1)
#possible values for atlas_mode= sliding \ stacked
atlas_mode='sliding'


#calculating surprises
df_dict_surprises =atlas.compute_surprises_for_df_dict(df_dict, prediction_method, keep, field,atlas_interval_duration, atlas_sliding_shift,atlas_mode)

pprint("fin du du dict surprise")

#calculating grubbs tests
df_dict_grubbs=atlas.compute_grubbs_for_surprises_df_dict(df_dict_surprises, prediction_method, keep,atlas_interval_duration, atlas_sliding_shift,atlas_mode)

pprint("fin du du df_dict_grubbs")
#inserting the Grubbs test result in mydb 
dataset_atlas.insert_grubbs_in_mydb(df_dict_grubbs, prediction_method, keep,atlas_interval_duration,atlas_sliding_shift, atlas_mode)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df_dict